# Imports

In [38]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
import optuna as opt

# Importing the data

In [8]:
# Import the data in ./../data/

# Categorically Encoding "class"

In [9]:
# Use pandas cat.codes to convert the categorical data to numerical data, this is because xgboost only accepts numerical data as 
# the input in classification problems.

In [10]:
# Define X and y

# Train_test_split 

In [1]:
# Split the data into train and test sets with test_size=0.2 and random_state=42

# Optuna hyperparameter optimalization

In [2]:
def objective(trial):
    param = {
            "n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
            'max_depth':trial.suggest_int('max_depth', 2, 25),
            'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
            'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
            'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
            'gamma':trial.suggest_int('gamma', 0, 5),
            'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
            'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
            'nthread' : -1
        }
    model = xgb.?(**param)
    model.fit(?, ?)
    y_pred = model.predict(?)
    return f1_score(?, ?, average='macro')


SyntaxError: invalid syntax (1939486727.py, line 14)

In [3]:
# Create a Optuna study object with .create_study() and set the direction to maximize, this is because we want to maximize the f1 score.
# Then, run the study with .optimize() and pass the objective function and the number of trials as parameters.
study = 
study.

SyntaxError: invalid syntax (3320903580.py, line 3)

# Using optuna hyperparams to make a model

In [4]:
# Retrieve the best parameters from the study with .best_params
# And create a XGBoost classifier with the best parameters, then fit the model with the training data

# Predicting on the test set with the model

In [5]:
# Predict the test data

# Using cross-validation to get an accurate score

In [6]:
# Use cross_val_score to get the cross validation score of the model with the training data.
# Cross validation is a technique to evaluate the performance of a model on unseen data, it is done by splitting the data into "k" folds
# and then training the model on "k-1" folds and testing it on the remaining fold. This is repeated "k" times and the average of the
# scores is taken as the final score. This effectively means that the data is trained and validated on all of the data.
# Run cross_val_score with 5 folds and scoring='f1_macro'

scores = cross_val_score(?, ?, ?, cv=?, scoring=?)

# Print the mean and standard deviation of the scores, this is a good way to evaluate the performance of the model.

SyntaxError: invalid syntax (3585096085.py, line 6)

# Plot the feature importances

In [7]:
# XGBoost has a built in function to plot the feature importance, use it to plot the feature importance of the model.
# Feature importance shows how much each feature contributes to the model, this is a good way to see which features are important

xgb.?

SyntaxError: invalid syntax (4249416720.py, line 4)

# Plotting the confusion matrix

In [8]:
# A confusion matrix is a table that is often used to describe the performance of a classification model (or "classifier") 
# on a set of test data for which the true values are known. The confusion matrix itself is relatively simple to understand,
# but the related terminology can be confusing. The confusion matrix shows the ways in which your classification model is confused
# when it makes predictions. It gives you insight not only into the errors being made by your classifier but more importantly
# the types of errors that are being made.

# Get the confusion matrix from sklearn.metric.confusion_metric and plot it with ConfusionMatrixDisplay
# Make sure to include display_labels with the class names (HINT: model.classes_) to make the plot more readable



cm = ?(?, ?)
disp = ?(?=?, ?=?)

disp.?(cmap='Blues')

SyntaxError: invalid syntax (3714684622.py, line 12)

# Plot a tree from the model

In [ ]:
# XGBoost is a tree based ensemble model, it is always a good idea to visualize the tree to see how the model is making the predictions.
# Use xgb.to_graphviz to visualize the tree and save it as a png file.